In [11]:
import argparse
import time
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from scipy import sparse
from dataset import *
from dataloader import *
from loss import *
from model import *
from torch.nn import Softmax
import os
import bottleneck as bn
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [12]:
if torch.cuda.is_available():
        cuda_aval = True

device = torch.device("cuda" if cuda_aval else "cpu")
print("Now using device : ", device)
###################
print("Load and Preprocess Movielens dataset")
# Load Data
uim = pd.read_csv("/opt/ml/input/workspace/CF/DL CF/Mult-VAE/new_user_movie_interaction.csv")
user_id = uim["user"].to_numpy()
uim_np = uim.to_numpy()[:, 1:]
print("Done!")

Now using device :  cuda
Load and Preprocess Movielens dataset
Done!


In [13]:
iiii = pd.read_csv("/opt/ml/input/data/train/pro_sg/unique_sid.txt", header=None)
show2id = dict((sid, i) for (i, sid) in enumerate(iiii))
item_id = iiii.to_numpy().reshape(6807)

In [14]:
#load model
f = open("model.pt", 'rb')
model  = torch.load(f).to(device)

In [15]:
model.eval()
result = list()
for i in range(31360) : 
    user = user_id[i]

    data_np = uim_np[i]

    data = torch.FloatTensor(data_np)
    data = data.reshape(1,-1).to(device)
    probability, _, _ = model(data)
    probability_np = np.array(probability.cpu().detach())[0]
    
    probability_np[data_np.nonzero()] = -np.inf
    idx = np.argpartition(probability_np, -10)[-10:]
    candidates = [[index, probability_np[index]] for index in idx]
    candidates.sort(key=lambda x:x[1], reverse = True)
    items = item_id[idx]
    for candidate in candidates :
        result.append((user, item_id[candidate[0]]))
    if i%5000 == 0 :
        print(f"{i}/31360 complete")

/opt/conda/lib/python3.8/site-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


0/31360 complete
5000/31360 complete
10000/31360 complete
15000/31360 complete
20000/31360 complete
25000/31360 complete
30000/31360 complete


In [16]:
info = pd.DataFrame(result, columns=['user','item'])
info.to_csv("submission_e150.csv",index=False)
print("Inference Done!")

Inference Done!
